In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import datetime
import openpyxl

In [ ]:
# 엑셀 파일 불러오기
filename = 'Wadiz_Sport_Mobility.xlsx'
data = pd.read_excel(filename)

In [ ]:
# 컬럼 추가 및 정리

# 1. 목표금액과기간에서 목표금액과 펀딩시작날짜, 펀딩마감날짜 추가 + 펀딩마감날짜부터 배송까지 기간 (마감배송차이) 추가
목표금액 = []
펀딩시작날짜 = []
펀딩마감날짜 = []
배송시작날짜 = []
마감배송차이 = []

for _ in range(0,len(data)):
    split_data = data.loc[_][['목표금액과기간']].str.split(' ')
    ls = split_data['목표금액과기간']
    
    # 목표금액 추출
    price = ls[2].replace(",","").replace("원","")
    목표금액.append(int(price))

    # 펀딩 시작, 마감 날짜 추출
    start_date, end_date = ls[8].split('-')
    펀딩시작날짜.append(start_date)
    펀딩마감날짜.append(end_date)
    
    # 첫 배송날짜 추출
    deliver_date = data.loc[_][['배송시작날짜']].str.split('/')
    배송시작날짜.append(deliver_date[0][0].replace(" ",""))
    
    # 첫 배송날짜와 펀딩마감 날짜의 차이 일수 계산
    oneDatetime = datetime.datetime.strptime(펀딩마감날짜[_], '%Y.%m.%d')
    twoDatetime = datetime.datetime.strptime(배송시작날짜[_], '%Y.%m.%d')
    result = twoDatetime - oneDatetime
    마감배송차이.append(result.days)
    
# dataframe에 추가 
data['목표금액'] = 목표금액
data['펀딩시작날짜'] = 펀딩시작날짜
data['펀딩마감날짜'] = 펀딩마감날짜
data['마감배송차이'] = 마감배송차이

# 목표금액과기간 컬럼 지우기
data.drop('목표금액과기간', axis=1, inplace=True)


# 2. 펀딩 시작 요일 추가
펀딩시작날짜 = pd.to_datetime(펀딩시작날짜)
data['펀딩시작요일'] = 펀딩시작날짜.strftime('%a')


# 3. 펀딩 성공 여부 ( '성공' vs '실패')
data['펀딩성공여부'] = data['달성률'].apply(lambda x: 1 if x>99 else 0)


# 4. 앵콜펀딩 여부 ('앵콜' vs '최초')
data['앵콜펀딩여부'] = data['제목'].apply(lambda x: 1 if  '앵콜' in x else 0)


# 5. 과거 프로젝트 수 컬럼에서 -1값들을 0으로 바꾸기
data['과거프로젝트수'] = data['과거프로젝트수'].apply(lambda x: 0 if x == -1 else x)


# 6. 과거 프로젝트 성공 개수 처리 (성공한 프로젝트일 경우 과거 프로젝트 성공 개수에서 1 빼기)
for index, row in data.iterrows():
    if row['펀딩성공여부'] == '성공':
        data.at[index,'과거성공프로젝트수'] = data.iloc[index]['과거성공프로젝트수']-1
        
# 6번의 과정에서 -1이 되어버린 값들을 0으로 바꾸기
data['과거성공프로젝트수'] = data['과거성공프로젝트수'].apply(lambda x: 0 if x == -1 else x)

        
# 7. 인스타 여부 컬럼추가
data['인스타존재여부'] = data['인스타팔로워수'].apply(lambda x: 1 if x in ['no account', 'link error'] else 0)


# 8. 인스타그램 no account와 link error값들을 0으로 바꾸기
data['인스타팔로워수'] = data['인스타팔로워수'].apply(lambda x: 0 if x in ['no account', 'link error'] else x)

In [ ]:
# 데이터 타입 변환

# 1. 날짜 데이터들 datatime 데이터로 변환
data['펀딩시작날짜'] = pd.to_datetime(펀딩시작날짜)
data['펀딩마감날짜'] = pd.to_datetime(펀딩마감날짜)
data['배송시작날짜'] = pd.to_datetime(배송시작날짜)

# 2. str 타입 숫자 타입으로 변환
data['달성액'] = data['달성액'].apply(lambda x: int(x.replace('원', '')))
data['서포터수'] = data['서포터수'].apply(lambda x: int(x.replace(',', '')))

In [ ]:
# 파일 저장
filename = filename.replace(".xlsx","_전처리.xlsx")
data.to_excel(filename)